In [8]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
EPOCHS = 10
LOGO_LABEL = 1  # Label for the logo class

# Define a pre-trained network as Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom model on top of Xception
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification (logo or not)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Create a sample dataset
# Here, a simple dataset is created for illustration purposes only.
# In a real project, it is important to create a larger and representative dataset.
X_train = np.random.rand(100, 299, 299, 3)
y_train = np.random.randint(2, size=(100,))

# Train the model
model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

# Save the trained model
model.save('logo_detection_model.h5')

# Example code for model evaluation
loss, accuracy = model.evaluate(X_train, y_train)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Example of loading the model
loaded_model = load_model('logo_detection_model.h5')

# Function for logo detection in a video
def logo_detection_in_video(video_path, model):
    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame for prediction
        frame = cv2.resize(frame, IMAGE_SIZE)
        frame = frame / 255.0  # Normalization
        frame = np.expand_dims(frame, axis=0)

        # Make a prediction using the trained model
        prediction = model.predict(frame)

        # Check if a logo is detected
        if prediction[0][0] > 0.5:  # Threshold value for logo detection
            print(f"Time when the logo is seen: {frame_count / cap.get(cv2.CAP_PROP_FPS)} seconds")

        frame_count += 1

    cap.release()

# Example usage
video_path = 'C:/Users/mehme/video.mp4'
logo_detection_in_video(video_path, loaded_model)


Epoch 1/10
4/4 [==============================] - 11s 2s/step - loss: 0.7394 - accuracy: 0.6100
Epoch 2/10
4/4 [==============================] - 8s 2s/step - loss: 1.0202 - accuracy: 0.4500
Epoch 3/10
4/4 [==============================] - 7s 2s/step - loss: 0.7800 - accuracy: 0.5500
Epoch 4/10
4/4 [==============================] - 8s 2s/step - loss: 0.8346 - accuracy: 0.5500
Epoch 5/10
4/4 [==============================] - 8s 2s/step - loss: 0.6850 - accuracy: 0.5200
Epoch 6/10
4/4 [==============================] - 9s 2s/step - loss: 0.7305 - accuracy: 0.4600
Epoch 7/10
4/4 [==============================] - 9s 2s/step - loss: 0.6552 - accuracy: 0.6700
Epoch 8/10
4/4 [==============================] - 9s 2s/step - loss: 0.6525 - accuracy: 0.6000
Epoch 9/10
4/4 [==============================] - 9s 2s/step - loss: 0.6653 - accuracy: 0.5900
Epoch 10/10
4/4 [==============================] - 9s 2s/step - loss: 0.6599 - accuracy: 0.5500
Test Loss: 0.6599
Test Accuracy: 0.5500
